# RMN Baseline

Training an RMN on sessions 105 - 111

In [1]:
import os
import pandas as pd

In [2]:
os.chdir("../../../scripts/assembly/")

In [3]:
from document import *
from subject import subject_keywords
from constant import DOC_PATH

In [5]:
# run several times
os.chdir("../modeling")
from token_mapping import *
from embeddings import *
from helper import load_pickled_object
from rmn import RMN

In [7]:
DOC_SAMPLE_PATH = os.path.join(DOC_PATH, "doc-sample/")
docs_df = load_documents(['alcohol', 'health'], DOC_SAMPLE_PATH)
# N = 50000
# docs_df = docs_df.sample(N)

In [8]:
N = 50
docs_df = docs_df.sample(N)

In [9]:
feature_columns = [
 'speakerid',
 'chamber',
 'state',
 'gender',
 'party',
 'congress',
 'subject']

# make tokenizer and metadata dicts
tokenizer_dict = build_tokenizer_dict(docs_df)
metadata_dict = build_metadata_dict(docs_df, feature_columns)
metadata_dict.keys()

dict_keys(['speakerid', 'chamber', 'state', 'gender', 'party', 'congress', 'subject'])

In [10]:
# Get embeddings
embedding_file = "/home/rocassius/gen-data/tools/embbedding_index_50d"
embeddings_index = load_pickled_object(embedding_file)
embeddings_matrix = build_embedding_matrix(tokenizer_dict['word_index'], embeddings_index).astype('float16')

# RMN Training

In [11]:
rmn = RMN()
rmn.embedding_matrix = embeddings_matrix
rmn.tokenizer_dict = tokenizer_dict
rmn.metadata_dict = metadata_dict
rmn.num_topics = 40
rmn.build_model()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [12]:
import sys

In [13]:
inputs = rmn.prep_inputs(docs_df)
y = inputs[0]

In [14]:
rmn.model.fit(x=inputs, y=y, epochs = 10, batch_size = 50)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 50 samples
Epoch 1/10
50/50 [==============================] - 2s 38ms/sample - loss: 8.0164
Epoch 2/10
50/50 [==============================] - 0s 90us/sample - loss: 8.0053
Epoch 3/10
50/50 [==============================] - 0s 75us/sample - loss: 8.0007
Epoch 4/10
50/50 [==============================] - 0s 75us/sample - loss: 8.0001
Epoch 5/10
50/50 [==============================] - 0s 75us/sample - loss: 8.0012
Epoch 6/10
50/50 [==============================] - 0s 76us/sample - loss: 8.0024
Epoch 7/10
50/50 [==============================] - 0s 69us/sample - loss: 8.0029
Epoch 8/10
50/50 [==============================] - 0s 86us/sample - loss: 8.0027
Epoch 9/10
50/50 [==============================] - 0s 64us/sample - loss: 8.0019
Epoch 10/10
50/50 [==============================] - 0s 71us/sample - loss: 8.0007


In [15]:
rmn.model

In [17]:
rmn.save_rmn("test", "/home/rocassius/gen-data/models")

In [18]:
import tensorflow as tf

In [19]:
tf.__version__

'1.15.0'

In [20]:
from tensorflow import keras

In [21]:
keras.__version__

'2.2.4-tf'